In [1]:
import os
import sys

from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
"""
loading GCP Big Query Client
"""
from google.cloud import bigquery

project_id = os.environ.get('GCP_PROJECT_ID')
table_name = os.environ.get('BG_TABLE_NAME')
dataset_id = os.environ.get('BG_DATASET_ID')
bg_location = os.environ.get('BG_LOCATION')
table_id = os.environ.get('BG_TABLE_ID')

# Construct a BigQuery client object.
client = bigquery.Client(project_id)

In [3]:
"""
Loading Pandas to validate some checks in sample data, to optimize time.
"""

from divvy_bike_share_data_analysis.data_loader import load_dataset_to_local_fs
from divvy_bike_share_data_analysis.utils_pyspark import load_data_to_spark_big_query_enabled

import glob

# Collecting data to local directory
DATA_COLLECTION_DIR: str = '../data_collection/'

# This is commented out, as I only collect the csvs in the very first run.
if not glob.glob(os.path.join(DATA_COLLECTION_DIR, '*.csv')):
    print('collecting data to local fs')
    load_dataset_to_local_fs(DATA_COLLECTION_DIR, [2020, 2021, 2022, 2023])

# creating pyspark session and pre-processing data
divvy_df = load_data_to_spark_big_query_enabled(DATA_COLLECTION_DIR)
divvy_df.dropna()

:: loading settings :: url = jar:file:/Users/macbook/code/TechProject1/venv/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/macbook/.ivy2/cache
The jars for the packages stored in: /Users/macbook/.ivy2/jars
com.google.cloud.spark#spark-bigquery-with-dependencies_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-aea1bd3f-e61d-4ab1-973a-e0d237ee7e71;1.0
	confs: [default]
	found com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.21.0 in central
:: resolution report :: resolve 178ms :: artifacts dl 8ms
	:: modules in use:
	com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.21.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	-------------------------------------------------------------

DataFrame[ride_id: string, rideable_type: string, started_at: timestamp, ended_at: timestamp, start_station_name: string, start_station_id: string, end_station_name: string, end_station_id: string, start_lat: float, start_lng: float, end_lat: float, end_lng: float, member_casual: string]

In [4]:
# from google.cloud import bigquery
# 
# schema=[
#     bigquery.SchemaField("ride_id", "STRING"),
#     bigquery.SchemaField("rideable_type", "STRING"),
#     bigquery.SchemaField("started_at", "TIMESTAMP"),
#     bigquery.SchemaField("ended_at", "TIMESTAMP"),
#     bigquery.SchemaField("start_station_name", "STRING"),
#     bigquery.SchemaField("start_station_id", "STRING"),
#     bigquery.SchemaField("end_station_name", "STRING"),
#     bigquery.SchemaField("end_station_id", "STRING"),
#     bigquery.SchemaField("start_lat", "FLOAT"),
#     bigquery.SchemaField("start_lng", "FLOAT"),
#     bigquery.SchemaField("end_lat", "FLOAT"),
#     bigquery.SchemaField("end_lng", "FLOAT"),
#     bigquery.SchemaField("member_casual", "STRING"),
# ]
# 
# table = client.get_table(table_id)  # Make an API request.
# original_schema = table.schema
# new_schema = original_schema[:]  # Creates a copy of the schema.
# [new_schema.append(field) for field in schema]
# table.schema = new_schema
# table = client.update_table(table, ["schema"])  # Make an API request.
 

In [5]:
"""
Pushing all data to BigQuery table
"""
divvy_df.write.format('bigquery').option('divvy_trips', 'x-oxygen-413115.divvy_project1').save()

Py4JJavaError: An error occurred while calling o54.save.
: com.google.cloud.spark.bigquery.repackaged.com.google.inject.ProvisionException: Unable to provision, see the following errors:

1) Error in custom provider, java.lang.IllegalArgumentException: No table has been specified
  at com.google.cloud.spark.bigquery.v2.SparkBigQueryConnectorModule.provideSparkBigQueryConfig(SparkBigQueryConnectorModule.java:68)
  while locating com.google.cloud.spark.bigquery.SparkBigQueryConfig

1 error
	at com.google.cloud.spark.bigquery.repackaged.com.google.inject.internal.InternalProvisionException.toProvisionException(InternalProvisionException.java:226)
	at com.google.cloud.spark.bigquery.repackaged.com.google.inject.internal.InjectorImpl$1.get(InjectorImpl.java:1097)
	at com.google.cloud.spark.bigquery.repackaged.com.google.inject.internal.InjectorImpl.getInstance(InjectorImpl.java:1131)
	at com.google.cloud.spark.bigquery.BigQueryRelationProvider.createRelation(BigQueryRelationProvider.scala:1

In [ ]:
"""
Once I split the loaded data into a sample size, I can use pandas to validate some checks, and save time as Spark is slow locally. 
"""

divvy_df = divvy_df.dropna()
sampled_df = divvy_df.sample(withReplacement=False, fraction=0.01)
# sampled_df.columns
divvy_df_pandas = sampled_df.toPandas()
from divvy_bike_share_data_analysis import bike_stations

bike_stations = bike_stations.get_unique_bike_stations_ids(sampled_df)

In [ ]:
grouped_stations_by_matching_id_pd = divvy_df_pandas[
    ['start_station_id', 'start_station_name', 'end_station_id', 'end_station_name']].drop_duplicates().dropna()
conflicting_stations = grouped_stations_by_matching_id_pd[
    grouped_stations_by_matching_id_pd['start_station_name'] != grouped_stations_by_matching_id_pd['end_station_name']]

bike_stations_pd = grouped_stations_by_matching_id_pd[["start_station_id", "start_station_name"]].rename(
    columns={"start_station_id": "station_id", "start_station_name": "station_name"}).drop_duplicates()


In [ ]:
from divvy_bike_share_data_analysis.bike_stations import categorize_time_of_day
from pyspark.sql.functions import udf, hour
from pyspark.sql.types import StringType

# Adding a categorical field to determine if the station is a start or end station
# Convert to a UDF Function by passing in the function and return type

# Before we start, check for nulls
# sampled_df.filter(sampled_df.started_at.isNull()).count() # there's 0

time_of_day_udf = udf(categorize_time_of_day, StringType())
sampled_df_with_day_period = sampled_df.withColumn('day_period', time_of_day_udf(hour('started_at')))
# sampled_df_with_day_period.show(5)
from pyspark.sql.functions import col

sampled_df_with_day_period.filter(col('day_period').isNull()).count()

In [ ]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="day_period", outputCol="day_period_index")
sampled_df_with_day_period_indexed = indexer.fit(sampled_df_with_day_period).transform(sampled_df_with_day_period)

#from pyspark.ml.feature import OneHotEncoder
# OneHotEnconding
# 
# encoder = OneHotEncoder(inputCols=["day_period_index"], outputCols=["day_period_ohe"])
# sampled_df_with_day_period_ohed = encoder.fit(sampled_df_with_day_period_indexed).transform(sampled_df_with_day_period_indexed)
# from pyspark.sql.functions import when
# 
# sampled_df_with_day_period_indexed.filter(col('day_period_index').isNull()).count()
# sampled_df_with_day_period_indexed = sampled_df_with_day_period_indexed.withColumn("day_period_index", when(col("day_period_index").isNull(), 0.0).otherwise(col("day_period_index"))) # TODO review this as potential problem. 




In [ ]:
"""
Pandas Approach:
divvy_df_pandas.dropna(subset=['start_station_id', 'end_station_id'], inplace=True)
divvy_df_pandas['start_station_name', 'end_station_name'] = divvy_df_pandas['start_station_id'].map(lambda x: bike_stations_pd[bike_stations_pd['station_id'] == x]['station_name'].values[0])
divvy_df_pandas[['start_station_id', 'start_station_name', 'end_station_id', 'end_station_name']]
"""
sanitized_bike_stations = sampled_df_with_day_period_indexed.join(
    bike_stations.withColumnRenamed("station_name", "start_station_name"),
    sampled_df_with_day_period_indexed.start_station_id == bike_stations.station_id,
    "left").drop("station_id")

# Join to map end_station_id to end_station_name
sanitized_bike_stations = sanitized_bike_stations.join(
    bike_stations.withColumnRenamed("station_name", "end_station_name"),
    sanitized_bike_stations.end_station_id == bike_stations.station_id,
    "left").drop("station_id")

In [ ]:
from pyspark.ml.feature import StringIndexer

indexer_start_station_id = StringIndexer(inputCol="start_station_id", outputCol="start_station_id_index")
indexer_end_station_id = StringIndexer(inputCol="end_station_id", outputCol="end_station_id_index")
sanitized_bike_stations_id_indexed = indexer_start_station_id.fit(sanitized_bike_stations).transform(
    sanitized_bike_stations)
sanitized_bike_stations_id_indexed = indexer_end_station_id.fit(sanitized_bike_stations).transform(
    sanitized_bike_stations_id_indexed)
sanitized_bike_stations_id_indexed.show()

In [ ]:
sanitized_bike_stations_id_indexed.dropna(subset=['day_period_index'])
sanitized_bike_stations_id_indexed.dropna(subset=['start_station_id_index'])

In [ ]:
from pyspark.ml.feature import VectorAssembler

# most popular stations by period of day
feature_cols = ['start_station_id_index', 'day_period_index']
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Transform the data to have the features column
adf_kmeans = assembler.transform(sanitized_bike_stations_id_indexed)
### Custom spark Session to support synapsesML:
import pyspark

spark = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:1.0.2") \
    .config("spark.jars.repositories", "https://mmlspark.azureedge.net/maven") \
    .getOrCreate()

from pyspark.ml.clustering import KMeans

# Number of clusters
k = 5  # Adjust based on your data and needs

kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
model = kmeans.fit(adf_kmeans)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Make predictions
# predictions = model.transform(adf_kmeans)
# 
# # Group by cluster label and start_station_id, and count occurrences
# popular_start_stations = predictions.groupBy("prediction", "start_station_id_index")
# 
# # Find the most popular station in each cluster
# most_popular_start_stations = popular_start_stations.groupBy("prediction").max("count")
# 
# popular_destinations = predictions.groupBy("prediction", "end_station_id_index").count()
# most_popular_destination = popular_start_stations.groupBy("prediction").max("count")

# TODO apply K-means clustering and to find the most popular source stations - Plot
# TODO - Can I apply K-means for most popular stations by time of day? - Plot
# TODO - most popular routes - Plot 


predictions.columns

plt.figure(figsize=(10, 6))
sns.scatterplot(data=predictions, x='start_station_id_index', y='start_lat', hue='prediction', palette='viridis')
# plt.title('KMeans Clustering Results')
# plt.show()